In [1]:
!pip install -U sentence_transformers
!pip install xformers
!pip install bitsandbytes
!pip install peft
!pip install huggingface_hub
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 93.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 104.4 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [2]:
import os
import json
import torch
from sentence_transformers import SentenceTransformer
#from langchain_community.document_loaders import PyPDFLoader
#from langchain_text_splitters import RecursiveCharacterTextSplitter

## Pretrained model
We choose `stella_en_400M_v5` model from [MTEB leaderboard](https://huggingface.co/spaces/mteb/leaderboard).

In [17]:
query_prompt_name = "s2p_query"
queries = [
    "What material is the rear wing of the 718 Cayman GT4 RS made of?",
    "What is the unique feature of the Cayenne Turbo GT?",
    "What customization options are available for the Taycan Cross Turismo?",
    "What is the combined CO₂ emissions for the 718 Cayman GTS 4.0?",
    "What is the GTS model?",
    "What makes the Cayenne iconic in its category?"
]

docs = [
    "Carbon fiber reinforced plastic (CFRP).",
    "It offers 471 kW (640 PS) and is optimized for high performance.",
    "Exclusive paint finishes, interior trims, and wheel designs.",
    "The 718 Cayman GTS 4.0 produces zero CO₂ emissions because it is fully electric.",
    "GTS model is a great coaching system for MCQ practicing, which G stands for Guardian, T for Teacher and S for Student.",
    "The Cayenne is iconic for being the capital of French Guiana, known for its colonial architecture and a thriving pepper trade."
]

# ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# on gpu
model = SentenceTransformer("dunzhang/stella_en_400M_v5", trust_remote_code=True).cuda()
# you can also use this model without the features of `use_memory_efficient_attention` and `unpad_inputs`. It can be worked in CPU.
# model = SentenceTransformer(
#     "dunzhang/stella_en_400M_v5",
#     trust_remote_code=True,
#     device="cpu",
#     config_kwargs={"use_memory_efficient_attention": False, "unpad_inputs": False}
# )
query_embeddings = model.encode(queries, prompt_name=query_prompt_name)
doc_embeddings = model.encode(docs)
print(query_embeddings.shape, doc_embeddings.shape)

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(6, 1024) (6, 1024)


## Fine-tuned model

In [19]:
model_path = "/kaggle/input/porschechallenge/models/models/stella_en_400M_v5/finetune_triplets_2025-01-02_18-06-49" 
fine_tuned_model = SentenceTransformer(
    model_path, 
    device="cuda" if torch.cuda.is_available() else "cpu", 
    trust_remote_code=True,
)
fine_tuned_query_embeddings = fine_tuned_model.encode(queries, prompt_name=query_prompt_name)
fine_tuned_doc_embeddings = fine_tuned_model.encode(docs)
print(fine_tuned_query_embeddings.shape, fine_tuned_doc_embeddings.shape)

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(6, 1024) (6, 1024)


## Comparison between fine-tuned model and pre-trained model
Issues of pretrained model: 
1. positive example sentence pairs but low similariy
2. negative example sentence pairs but high similariy
After finetuning, the fine-tuned model can improve the performance.

### Case 1: Positive answer but low similarity score
#### Pretrained model

In [ ]:
for i in range(3):
    similarities = model.similarity(query_embeddings[i], doc_embeddings[i])
    print(f"Uesr Query: {queries[i]}")
    print(f"Answer: {docs[i]}")
    print(f"Similarity score: {similarities.data.cpu().numpy()[0][0]} \n\n")

Uesr Query: What material is the rear wing of the 718 Cayman GT4 RS made of?
Answer: Carbon fiber reinforced plastic (CFRP).
Similarity score: 0.5579700469970703 


Uesr Query: What is the unique feature of the Cayenne Turbo GT?
Answer: It offers 471 kW (640 PS) and is optimized for high performance.
Similarity score: 0.41068941354751587 


Uesr Query: What customization options are available for the Taycan Cross Turismo?
Answer: Exclusive paint finishes, interior trims, and wheel designs.
Similarity score: 0.5345098376274109 




#### Fine-tuned model

In [ ]:
for i in range(3):
    similarities = fine_tuned_model.similarity(fine_tuned_query_embeddings[i], fine_tuned_doc_embeddings[i])
    print(f"Uesr Query: {queries[i]}")
    print(f"Answer: {docs[i]}")
    print(f"Similarity score: {similarities.data.cpu().numpy()[0][0]} \n\n")

Uesr Query: What material is the rear wing of the 718 Cayman GT4 RS made of?
Answer: Carbon fiber reinforced plastic (CFRP).
Similarity score: 0.827846884727478 


Uesr Query: What is the unique feature of the Cayenne Turbo GT?
Answer: It offers 471 kW (640 PS) and is optimized for high performance.
Similarity score: 0.7782524824142456 


Uesr Query: What customization options are available for the Taycan Cross Turismo?
Answer: Exclusive paint finishes, interior trims, and wheel designs.
Similarity score: 0.774215042591095 




### Case 2: negative answer but high similariy
#### Pretrained model

In [ ]:
for i in range(3, 6):
    similarities = model.similarity(query_embeddings[i], doc_embeddings[i])
    print(f"Uesr Query: {queries[i]}")
    print(f"Answer: {docs[i]}")
    print(f"Similarity score: {similarities.data.cpu().numpy()[0][0]} \n\n")

Uesr Query: What is the combined CO₂ emissions for the 718 Cayman GTS 4.0?
Answer: The 718 Cayman GTS 4.0 produces zero CO₂ emissions because it is fully electric.
Similarity score: 0.8328069448471069 


Uesr Query: What is the GTS model?
Answer: GTS model is a great coaching system for MCQ practicing, which G stands for Guardian, T for Teacher and S for Student.
Similarity score: 0.7170214056968689 


Uesr Query: What makes the Cayenne iconic in its category?
Answer: The Cayenne is iconic for being the capital of French Guiana, known for its colonial architecture and a thriving pepper trade.
Similarity score: 0.6261851787567139 




#### Fine-tuned model

In [20]:
for i in range(3, 6):
    similarities = fine_tuned_model.similarity(fine_tuned_query_embeddings[i], fine_tuned_doc_embeddings[i])
    print(f"Uesr Query: {queries[i]}")
    print(f"Answer: {docs[i]}")
    print(f"Similarity score: {similarities.data.cpu().numpy()[0][0]} \n\n")

Uesr Query: What is the combined CO₂ emissions for the 718 Cayman GTS 4.0?
Answer: The 718 Cayman GTS 4.0 produces zero CO₂ emissions because it is fully electric.
Similarity score: 0.21693462133407593 


Uesr Query: What is the GTS model?
Answer: GTS model is a great coaching system for MCQ practicing, which G stands for Guardian, T for Teacher and S for Student.
Similarity score: 0.5077574849128723 


Uesr Query: What makes the Cayenne iconic in its category?
Answer: The Cayenne is iconic for being the capital of French Guiana, known for its colonial architecture and a thriving pepper trade.
Similarity score: 0.4031425416469574 


